# This notebook compares sequence similarity between groups of TEs
- this a preparation notebook to :
    + get sequence from genome location given the mm39 ref genome
    + extract groups of TEs to output to fastas
    + use tools to get similarity score within each group
 
- see [notion write up page](https://www.notion.so/Sequence-similarity-comparison-20bd8266960180c59a9cdcf2852533c4?source=copy_link) for detailed comparison design
- this notebook computed the similarity scores for the top 10, 20, 50 and last 50 up regulated elements in TJP2 mutant organoids data

In [17]:
from pyfaidx import Fasta
import pandas as pd
from Bio.Seq import Seq
from pathlib import Path

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO, pairwise2
from itertools import combinations
from multiprocessing import Pool, cpu_count
from pathlib import Path

In [3]:
#from pairwise2 import format_alignment

In [4]:
import time

In [5]:
import sequence_comparison as seqComp

## functions

### in the seqComp file

In [62]:
def extract_ucsc_bedlike_regions(
    genome_fasta: str,
    regions_df: pd.DataFrame,
    output_fasta: str,
):
    """
    Extracts sequences from a UCSC-style genome FASTA using 0-based, end-exclusive coordinates.
    Assumes the region file has 4-5 columns in the order: chrom, start, end, strand, [optional name]
    """
    genome = Fasta(genome_fasta)
    #df = pd.read_csv(regions_file, sep="\t", header=0 if has_header else None)
    df = regions_df

    if df.shape[1] < 4:
        raise ValueError("Expected at least 4 columns: chrom, start, end, strand")

    chrom_col, start_col, end_col, strand_col = df.columns[:4]
    id_col = df.columns[4] if df.shape[1] > 4 else None

    with open(output_fasta, "w") as out_fasta:
        for idx, row in df.iterrows():
            chrom = str(row[chrom_col])
            start = int(row[start_col])
            end = int(row[end_col])
            strand = row[strand_col].strip()
            seq_id = row[id_col] if id_col else f"{chrom}:{start}-{end}({strand})"

            try:
                # UCSC-style: 0-based start, end-exclusive
                seq = genome[chrom][start:end].seq
                if strand == "-":
                    seq = str(Seq(seq).reverse_complement())
                elif strand != "+":
                    print(f"Warning: Unknown strand '{strand}' at row {idx}, treating as '+'")

                out_fasta.write(f">{seq_id}\n{seq}\n")

            except KeyError:
                print(f"Warning: Chromosome {chrom} not found in genome FASTA.")
            except Exception as e:
                print(f"Error at {chrom}:{start}-{end} ({strand}): {e}")

    print(f"✅ Extracted sequences saved to: {output_fasta}")

visualize some function component

In [18]:
seqA = "AccgTAGAG"
seqB = "acgag"
alignment = pairwise2.align.globalxx("ACCGTAGAG", "ACGAG", one_alignment_only = True)
alignment[0]

Alignment(seqA='ACCGTAGAG', seqB='A-C---GAG', score=5.0, start=0, end=9)

In [19]:
print( pairwise2.format_alignment( *alignment[0] ) )

ACCGTAGAG
| |   |||
A-C---GAG
  Score=5



In [23]:
# check the data type of the alignment
formatted_alignment = pairwise2.format_alignment( *alignment[0] )
type( formatted_alignment)

str

In [22]:
print( formatted_alignment )

ACCGTAGAG
| |   |||
A-C---GAG
  Score=5



In [45]:
matches = sum(res1 == res2 for res1, res2 in zip(alignment[0].seqA, alignment[0].seqB))

In [43]:
zip(alignment[0].seqA, alignment[0].seqB)

In [40]:
matches

5

In [48]:
sequences = [ ('t1', 'abcde'), ('t2', 'abcadae'),('t3', 'knkn'),('t4', 'giuugu')]

In [49]:
[seq_id for seq_id, _ in sequences]

['t1', 't2', 't3', 't4']

In [47]:
def load_sequences(fasta_path):
    return [(record.id, str(record.seq)) for record in SeqIO.parse(fasta_path, "fasta")]

def percent_identity(args):
    """ This is the key funtion to compute overlap between two given sequences: 
            Compute number of aligned bases
            Calculate pct alignment ( #aligned / max len of the seqs)
    @param args: id1, seq1, id2, seq2 in a tuple
    @return (id1, id2, identity, alignment_str): the TE ids, the pct alignment score, and the formatted alignment string """
    
    id1, seq1, id2, seq2 = args
    # this alignment step is not case sensitive though
    alignment = pairwise2.align.globalxx(seq1, seq2, one_alignment_only=True)[0] # only get one alignment result

    # calculate the match scores
    matches = sum(res1 == res2 for res1, res2 in zip(alignment.seqA, alignment.seqB))
    identity = matches / max(len(seq1), len(seq2)) * 100

    #extract the alignment string
    alignment_str = pairwise2.format_alignment( *alignment)
    
    return (id1, id2, identity, alignment_str)

def compute_pairwise_identities(sequences, n_jobs=4):
    """Wrapper function to call percent_identity in a parallele computing way """

    # get a list of tuples of two sequences and their IDs for all combination of 2 sequences within a group of TEs
    args = [(id1, seq1, id2, seq2) for (id1, seq1), (id2, seq2) in combinations(sequences, 2)]

    # uses multi-thread processing
    with Pool(processes=n_jobs) as pool:
        results = pool.map(percent_identity, args) # pool results from multiple calls of the function and inputs 
    return results

def generate_heatmap(pairwise_identities, sequence_ids, output_path):
    df = pd.DataFrame(index=sequence_ids, columns=sequence_ids, dtype=float)
    for id1, id2, identity in pairwise_identities.values:
        df.loc[id1, id2] = identity
        df.loc[id2, id1] = identity
        df.loc[id1, id1] = 100
        df.loc[id2, id2] = 100
    sns.clustermap(df.fillna(100), cmap="viridis", figsize=(10, 10), annot=False)
    plt.title("Pairwise Sequence Identity")
    plt.savefig(output_path, bbox_inches='tight')
    plt.close()



def process_group(fasta_path, out_dir, n_jobs=None, plot_heatmap = False):
    """ This is the calling function that compares sequence similarity and save the result to an specified out directory """
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    sequences = load_sequences(fasta_path)
    print(f"Loaded {len(sequences)} sequences from {fasta_path}")

    pairwise_results = compute_pairwise_identities(sequences, n_jobs or cpu_count())
    identities = [identity for _, _, identity, _ in pairwise_results]
    
    mean_identity = np.mean(identities)
    std_identity = np.std(identities)
    
    print(f"Average identity: {mean_identity:.2f}% ± {std_identity:.2f}%")

    # Save to CSV
    df = pd.DataFrame(pairwise_results, columns=["ID1", "ID2", "PercentIdentity", "Alignment_str"])
    df.to_csv(out_dir / f"{Path(fasta_path).stem}_pairwise_identity.csv", index=False)

    # Heatmap
    if( plot_heatmap):
        sequence_ids = [seq_id for seq_id, _ in sequences]
        heatmap_path = out_dir / f"{Path(fasta_path).stem}_heatmap.png"
        heatmap_arg1= df.iloc[:, 0:3]
        # this stdout is for debugging 
        #print( f"debugging: heatmap argument has the dimension of {heatmap_arg1.shape}")
        generate_heatmap(heatmap_arg1, sequence_ids, heatmap_path)

    return mean_identity, std_identity


### new functions

In [96]:
def alignment_str_parser( aln, line_width = 50, ):
    """ This function prints out the aligned sequences given a display width
    @param aln: this should be the alignment string list. Extracted from the pairwise2 comparison formatted alignment string sep by newline to a list
    @param line_width: how mang bases to display in one line
    @return None, this function print out all the alignments"""
    SEQ1_IND = 0
    ALI_IND = 1
    SEQ2_IND = 2
    NUM_COMPONENT = 3 # number of component: [ seq 1, seq 2 , the aligment vertical lines ]
    SCORE_IND = 3

    aln_width = max( len( aln[SEQ1_IND]), len( aln[SEQ2_IND]) )
    num_line = aln_width//line_width
    remainder = aln_width%line_width
    print( f"DEBUG: aln_width = {aln_width}, num_line = {num_line}, remainder = {remainder}")

    line_label_ls = ['TOP','MAT', 'BOT' ]
    
    if( num_line < 1 ):
        #print( 'DEBUG: short seq')
        for c in range( NUM_COMPONENT):
            print( aln[c])
    else:
        #print( 'DEBUG: long seq')
        for l in range( num_line):
            for c in range( NUM_COMPONENT):
                curr_line_start = l*line_width
                curr_line_end = (l+1)*line_width
                
                prefix = line_label_ls[c] + str( curr_line_start+1 ) +  ": "
                
                print( prefix + aln[c][curr_line_start: curr_line_end] )
            print("")
        for c2 in range( NUM_COMPONENT):
            last_line_start = num_line* line_width
            last_line_end = last_line_start + remainder
            
            prefix = line_label_ls[c] + str( last_line_start+1 ) + ": "
            
            print( prefix +  aln[c2][last_line_start: last_line_end ] )

    print( aln[SCORE_IND] )
    

In [86]:
def add_seq_comparison_info( comp_df, deseq_df, merged_result = False):
    """ This function add sequence length information and a adjusted score based on sequence length differences.
    An adjusted score: adj_percentIdentity will be calculated, where the formula is total_match / mean( len( seq1), len(seq2) )
    @param comp_df: the sequence comparison df that contains pairwise comparison result
    @param deseq_df: the DESeq result that contains the genome location 
    @return a modified comp_df that contains the additional information
    """

    # need: seq1 len, seq2 len, total score, new_avg_score
    # declare a return df
    ret_df = pd.DataFrame( columns=[ 'ID1', 'ID2', 'seq1_len' , 'seq2_len' , 'total_match' , 'adj_percentIdentity' ], index = comp_df.index )
    
    TOTAL_MATCH_LINE = -2 # this is the index to find total match score from the alignment str
    
    for ind in comp_df.index:
        #extract info from the comp df
        id1, id2, pctIden, aln = comp_df.loc[ind].values

        #get sequence length
        seq1_len = int( deseq_df.loc[id1, 'genoEnd']) - int( deseq_df.loc[id1, 'genoStart'] )
        seq2_len = int( deseq_df.loc[id2, 'genoEnd']) - int( deseq_df.loc[id2, 'genoStart'] )

        #get total match score
        total_score_str = aln.split("\n")[TOTAL_MATCH_LINE]
        total_score_str = total_score_str.split("=")[1]
        total_match = int( total_score_str )

        # calculate new score
        adj_score = total_match / (sum( [seq1_len, seq2_len ] )*0.5)
        adj_score = 100* adj_score

        #update the result df
        ret_df.loc[ind] = [ id1, id2, seq1_len, seq2_len, total_match, adj_score ] 

    if( merged_result):
        ret_df = pd.merge( left= comp_df, right= ret_df, how = 'left' )
    
    return ret_df
        

    

## load data

In [6]:
squire_sig_up = pd.read_csv("./compare_dTE_overlap/squire_sig_up.csv", header = 0, index_col = 0 )
squire_sig_up.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand
279_RLTR4_MM-int,chr7|116034016|116036134|RLTR4_MM-int:LTR:LTR|...,13.232138,7.261792,1.103073,6.583238,6.866753e-09,8.559487e-07,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_33,RLTR4_MM-int,LTR,LTR,chr7,116034016,116036134,+
845_L1MC2,"chr5|96975368|96975991|L1MC2:L1:LINE|284|-,.",84.993681,2.143785,0.213613,10.035838,4.290302e-08,4.946900e-06,TE,L1MC2:L1:LINE,L1MC2:L1:LINE_284,L1MC2,L1,LINE,chr5,96975368,96975991,-
1028_RMER17B2,"chr13|4307360|4308212|RMER17B2:ERVK:LTR|222|+,.",57.907641,2.313474,0.250347,9.241055,7.746373e-08,8.620185e-06,TE,RMER17B2:ERVK:LTR,RMER17B2:ERVK:LTR_222,RMER17B2,ERVK,LTR,chr13,4307360,4308212,+
1229_IAPLTR1_Mm,"chr6|28924173|28924528|IAPLTR1_Mm:ERVK:LTR|92|-,.",22.575679,7.303348,1.085208,6.729908,3.153163e-09,4.008132e-07,TE,IAPLTR1_Mm:ERVK:LTR,IAPLTR1_Mm:ERVK:LTR_92,IAPLTR1_Mm,ERVK,LTR,chr6,28924173,28924528,-
1401_RLTR4_Mm,"chr15|76450505|76451202|RLTR4_Mm:ERV1:LTR|14|-,.",49.190795,7.103274,1.107265,6.415154,1.773626e-08,2.128352e-06,TE,RLTR4_Mm:ERV1:LTR,RLTR4_Mm:ERV1:LTR_14,RLTR4_Mm,ERV1,LTR,chr15,76450505,76451202,-


In [7]:
squire_sig_up.columns

Index(['TE_ID', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue',
       'padj', 'element_type', 'TE_name', 'TE_index', 'repName', 'repFamily',
       'repClass', 'genoName', 'genoStart', 'genoEnd', 'strand'],
      dtype='object')

In [8]:
squire_sig_up['df_index_repName'] = squire_sig_up.index

## Testing small samples

### get squence from genome location:
- test case passed

In [7]:
#make region_df
test_region_df = squire_sig_up.head()[[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName']]

In [9]:
test_region_df

,genoName,genoStart,genoEnd,strand,df_index_repName
279_RLTR4_MM-int,chr7,116034016,116036134,+,279_RLTR4_MM-int
845_L1MC2,chr5,96975368,96975991,-,845_L1MC2
1028_RMER17B2,chr13,4307360,4308212,+,1028_RMER17B2
1229_IAPLTR1_Mm,chr6,28924173,28924528,-,1229_IAPLTR1_Mm
1401_RLTR4_Mm,chr15,76450505,76451202,-,1401_RLTR4_Mm


In [8]:
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'

In [11]:
# testt usage
extract_ucsc_bedlike_regions(
    genome_fasta= mm39_ref_path,       # UCSC-style combined FASTA with >chr1, >chr2, etc.
    regions_df= test_region_df,       # BED-like file: chr, start, end, strand, [name]
    output_fasta="test_group.fasta",
    has_header=False                         # Set to True if your TSV file has headers
)

# this actually works

✅ Extracted sequences saved to: test_group.fasta


In [12]:
test_region_df2 = squire_sig_up.iloc[ 4:10][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName']]

In [13]:
test_region_df2

,genoName,genoStart,genoEnd,strand,df_index_repName
1401_RLTR4_Mm,chr15,76450505,76451202,-,1401_RLTR4_Mm
1604_RLTR4_MM-int,chr15,76448211,76450505,-,1604_RLTR4_MM-int
1762_RLTR4_MM-int,chr1,164056220,164056423,+,1762_RLTR4_MM-int
1780_RLTR4_MM-int,chr1,182085381,182092968,+,1780_RLTR4_MM-int
1837_RLTR4_MM-int,chr11,5956407,5958777,+,1837_RLTR4_MM-int
1891_RLTR4_MM-int,chr1,164054347,164055989,+,1891_RLTR4_MM-int


In [14]:
# testt usage
extract_ucsc_bedlike_regions(
    genome_fasta= mm39_ref_path,       # UCSC-style combined FASTA with >chr1, >chr2, etc.
    regions_df= test_region_df2,       # BED-like file: chr, start, end, strand, [name]
    output_fasta="test_group2.fasta",
    has_header=False                         # Set to True if your TSV file has headers
)

# this actually works

✅ Extracted sequences saved to: test_group2.fasta


### compare sequence similarity

In [18]:
# Example usage:
process_group("test_group1.fasta", out_dir="test_result", n_jobs=4)
#process_group("test_group2.fasta", out_dir="test_result", n_jobs=4)

Loaded 5 sequences from test_group1.fasta
Average identity: 40.61% ± 12.53%


(40.60884004895388, 12.534331263094568)

In [19]:
process_group("test_group2.fasta", out_dir="test_result", n_jobs=4)

Loaded 6 sequences from test_group2.fasta
Average identity: 29.52% ± 22.14%


(29.520296757626912, 22.13635876036034)

## Comparing real result

- Get scores for the conditions of:
    + top10, top20, top50, top 50-100, and randomly selected 3 sets of 50 elements
    + and also compute the overlapped between the two methods

### test a larger group and count time
- test squire top 50

In [9]:
squire_sig_up.shape

(74, 18)

In [10]:
squire_sig_up.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand,df_index_repName
279_RLTR4_MM-int,chr7|116034016|116036134|RLTR4_MM-int:LTR:LTR|...,13.232138,7.261792,1.103073,6.583238,6.866753e-09,8.559487e-07,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_33,RLTR4_MM-int,LTR,LTR,chr7,116034016,116036134,+,279_RLTR4_MM-int
845_L1MC2,"chr5|96975368|96975991|L1MC2:L1:LINE|284|-,.",84.993681,2.143785,0.213613,10.035838,4.290302e-08,4.946900e-06,TE,L1MC2:L1:LINE,L1MC2:L1:LINE_284,L1MC2,L1,LINE,chr5,96975368,96975991,-,845_L1MC2
1028_RMER17B2,"chr13|4307360|4308212|RMER17B2:ERVK:LTR|222|+,.",57.907641,2.313474,0.250347,9.241055,7.746373e-08,8.620185e-06,TE,RMER17B2:ERVK:LTR,RMER17B2:ERVK:LTR_222,RMER17B2,ERVK,LTR,chr13,4307360,4308212,+,1028_RMER17B2
1229_IAPLTR1_Mm,"chr6|28924173|28924528|IAPLTR1_Mm:ERVK:LTR|92|-,.",22.575679,7.303348,1.085208,6.729908,3.153163e-09,4.008132e-07,TE,IAPLTR1_Mm:ERVK:LTR,IAPLTR1_Mm:ERVK:LTR_92,IAPLTR1_Mm,ERVK,LTR,chr6,28924173,28924528,-,1229_IAPLTR1_Mm
1401_RLTR4_Mm,"chr15|76450505|76451202|RLTR4_Mm:ERV1:LTR|14|-,.",49.190795,7.103274,1.107265,6.415154,1.773626e-08,2.128352e-06,TE,RLTR4_Mm:ERV1:LTR,RLTR4_Mm:ERV1:LTR_14,RLTR4_Mm,ERV1,LTR,chr15,76450505,76451202,-,1401_RLTR4_Mm


In [11]:
squire_sig_up.sort_values( by= 'log2FoldChange', ascending= False , inplace= True )

In [12]:
squire_sig_up.head()

,TE_ID,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,element_type,TE_name,TE_index,repName,repFamily,repClass,genoName,genoStart,genoEnd,strand,df_index_repName
9437_RLTR4_MM-int,chr13|22000000|22003430|RLTR4_MM-int:LTR:LTR|2...,2133.943181,14.594942,1.021194,14.292037,9.752864e-41,4.978605e-38,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_23,RLTR4_MM-int,LTR,LTR,chr13,22000000,22003430,+,9437_RLTR4_MM-int
11292_RLTR4_Mm,"chr13|22003625|22004366|RLTR4_Mm:ERV1:LTR|32|+,.",571.993187,12.694959,1.026310,12.369514,2.210350e-30,8.998084e-28,TE,RLTR4_Mm:ERV1:LTR,RLTR4_Mm:ERV1:LTR_32,RLTR4_Mm,ERV1,LTR,chr13,22003625,22004366,+,11292_RLTR4_Mm
12455_RLTR4_Mm,"chr13|21996579|21997320|RLTR4_Mm:ERV1:LTR|32|+,.",440.726573,12.319839,1.027687,11.987932,1.619963e-28,5.988278e-26,TE,RLTR4_Mm:ERV1:LTR,RLTR4_Mm:ERV1:LTR_32,RLTR4_Mm,ERV1,LTR,chr13,21996579,21997320,+,12455_RLTR4_Mm
4915_RLTR4_MM-int,chr13|22003422|22003625|RLTR4_MM-int:LTR:LTR|5...,321.002608,11.862092,1.023460,11.590188,1.294914e-26,4.430260e-24,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_5,RLTR4_MM-int,LTR,LTR,chr13,22003422,22003625,+,4915_RLTR4_MM-int
4128_RLTR4_MM-int,chr13|21997320|22000000|RLTR4_MM-int:LTR:LTR|1...,977.486167,9.566972,0.438444,21.820302,2.533456e-85,2.628257e-82,TE,RLTR4_MM-int:LTR:LTR,RLTR4_MM-int:LTR:LTR_13,RLTR4_MM-int,LTR,LTR,chr13,21997320,22000000,+,4128_RLTR4_MM-int


In [13]:
squire_top50_df = squire_sig_up.iloc[0:50][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ] 

In [14]:
squire_top50_df.head(2)

,genoName,genoStart,genoEnd,strand,df_index_repName
9437_RLTR4_MM-int,chr13,22000000,22003430,+,9437_RLTR4_MM-int
11292_RLTR4_Mm,chr13,22003625,22004366,+,11292_RLTR4_Mm


In [15]:
squire_top50_df.shape

(50, 5)

<font color = green>Get fasta for the group

In [18]:
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'
seqComp.extract_ucsc_bedlike_regions(
    genome_fasta= mm39_ref_path,       # UCSC-style combined FASTA with >chr1, >chr2, etc.
    regions_df= squire_top50_df,       # BED-like file: chr, start, end, strand, [name]
    output_fasta="squire_top50_df.fasta",
)

✅ Extracted sequences saved to: squire_top50_df.fasta


In [19]:
# test formatting float numbers
# fl = 1.2342342
# f"Time used: {fl:.2f} min"

<font color = green> get the sequence comparison result

In [17]:
TIME_S = time.time()
seqComp.process_group("squire_top50_df.fasta", out_dir="test_result", n_jobs=4,plot_heatmap=True )
TIME_E = time.time()
TOTAL_TIME = (TIME_E - TIME_S)/60
print( f"Time used: {TOTAL_TIME:.1f} min" )

Loaded 50 sequences from squire_top50_df.fasta
Average identity: 31.57% ± 24.13%
Time used: 2.5 min


### compute results for other conditions
- top10, top20, last 50

In [22]:
squire_sig_up.sort_values( by= 'log2FoldChange', ascending= False , inplace= True )

In [ ]:
squire_sig_up.head()

In [23]:
squire_top10_df = squire_sig_up.iloc[0:10][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ] 
squire_top20_df = squire_sig_up.iloc[0:20][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ]
squire_last50_df = squire_sig_up.iloc[-50::][[ 'genoName', 'genoStart', 'genoEnd', 'strand', 'df_index_repName'] ] 

In [19]:
# top 10 df
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= squire_top10_df,      
    output_fasta="squire_top10_df.fasta",
)

✅ Extracted sequences saved to: squire_top10_df.fasta


In [24]:
# top 20 df
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= squire_top20_df,      
    output_fasta="squire_top20_df.fasta",
)

# last 50 df
mm39_ref_path = '/home/lucy/Dropbox/mouse_ref_genome_mm39/mm39.fa'
seqComp.extract_ucsc_bedlike_regions( 
    genome_fasta= mm39_ref_path,     
    regions_df= squire_last50_df,      
    output_fasta="squire_last50_df.fasta",
)


✅ Extracted sequences saved to: squire_top20_df.fasta
✅ Extracted sequences saved to: squire_last50_df.fasta


In [19]:
TIME_S = time.time()
seqComp.process_group("squire_top10_df.fasta", out_dir="test_result", n_jobs=4,plot_heatmap=True )
TIME_E = time.time()
TOTAL_TIME = (TIME_E - TIME_S)/60
print( f"Time used: {TOTAL_TIME:.1f} min" )

Loaded 10 sequences from squire_top10_df.fasta
Average identity: 29.14% ± 23.45%
Time used: 0.2 min


In [25]:
# top 20 elements
TIME_S = time.time()
seqComp.process_group("squire_top20_df.fasta", out_dir="test_result", n_jobs=4,plot_heatmap=True )
TIME_E = time.time()
TOTAL_TIME = (TIME_E - TIME_S)/60
print( f"Time used: {TOTAL_TIME:.1f} min" )


Loaded 20 sequences from squire_top20_df.fasta
Average identity: 30.92% ± 26.19%
Time used: 0.3 min


In [26]:
# last 50 elements
TIME_S = time.time()
seqComp.process_group("squire_last50_df.fasta", out_dir="test_result", n_jobs=4,plot_heatmap=True )
TIME_E = time.time()
TOTAL_TIME = (TIME_E - TIME_S)/60
print( f"Time used: {TOTAL_TIME:.1f} min" )

Loaded 50 sequences from squire_last50_df.fasta
Average identity: 30.86% ± 20.97%
Time used: 1.3 min


## View result

In [41]:
squire_top10_result = pd.read_csv("./test_result/squire_top10_df_pairwise_identity.csv", header = 0 , index_col = None)
squire_top10_result.head()

,ID1,ID2,PercentIdentity,Alignment_str
0,9437_RLTR4_MM-int,11292_RLTR4_Mm,21.603499,gagactgttggaccagggaatactggtaccctgccagtccccctgg...
1,9437_RLTR4_MM-int,12455_RLTR4_Mm,21.603499,gagactgttggaccagggaatactggtaccctgccagtccccctgg...
2,9437_RLTR4_MM-int,4915_RLTR4_MM-int,5.918367,gagactgttggaccagggaatactggtaccctgccagtccccctgg...
3,9437_RLTR4_MM-int,4128_RLTR4_MM-int,56.180758,gagactgttggaccagggaatactggtaccctg-cc-ag-tccccc...
4,9437_RLTR4_MM-int,7290_RLTR4_MM-int,44.418084,---g-ag-----act-g---tt-gga--cc-------aggga--a-...


In [14]:
squire_top10_result.shape

(45, 4)

In [42]:
#pairwise2.format_alignment( squire_top10_result.iloc[0,3] )

### check and parse result format here

In [43]:
aln1 = squire_top10_result.loc[0, 'Alignment_str']

In [44]:
aln_ls = aln1.split("\n")
len( aln_ls )

5

In [45]:
for i in range( len( aln_ls)):
    print( aln_ls[i][0: min( 5, len( aln_ls[i] ) )] )

gagac
     
-----
  Sco



In [27]:
#display.max_colwidth

In [71]:
squire_top10_result.columns

Index(['ID1', 'ID2', 'PercentIdentity', 'Alignment_str'], dtype='object')

In [72]:
squire_sig_up.columns

Index(['TE_ID', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue',
       'padj', 'element_type', 'TE_name', 'TE_index', 'repName', 'repFamily',
       'repClass', 'genoName', 'genoStart', 'genoEnd', 'strand',
       'df_index_repName'],
      dtype='object')

In [60]:
alignment_str_parser( aln_ls, line_width = 100, )

DEBUG: aln_width = 3430, num_line = 34, remainder = 30
TOP1: gagactgttggaccagggaatactggtaccctgccagtccccctggaacacgcccctgctacccgttaagaaaccagggactaatgattacaggcctgtc
MAT1:                                                                                                     
BOT1: ----------------------------------------------------------------------------------------------------

TOP101: caagatctgagagaagtcaacaagcgggtggaagacatccaccccaccgtgcccaacccttacaacctcttgagcgggctcccaccgtcccaccagtggt
MAT101:                                                                                                     
BOT101: ----------------------------------------------------------------------------------------------------

TOP201: acactgtgcttgacttaaaggatgcctttttctgcctgagactccaccccaccagtcagcctctcttcgcctttgagtggagagacccagagatgggaat
MAT201:                                                                                                     
BOT201: ---------------------------------------------------------------------

In [67]:
squire_sig_up.loc['11292_RLTR4_Mm'][['genoName', 'genoStart' , 'genoEnd']]

genoName        chr13
genoStart    22003625
genoEnd      22004366
Name: 11292_RLTR4_Mm, dtype: object

In [68]:
squire_sig_up.loc['12455_RLTR4_Mm'][['genoName', 'genoStart' , 'genoEnd']]

genoName        chr13
genoStart    21996579
genoEnd      21997320
Name: 12455_RLTR4_Mm, dtype: object

In [69]:
22003625 - 21996579

7046

In [64]:
squire_top10_result.sort_values( by = 'PercentIdentity', ascending= False)

,ID1,ID2,PercentIdentity,Alignment_str
9,11292_RLTR4_Mm,12455_RLTR4_Mm,100.000000,tgaaagaccccaccataaggcttagcaagctagctgcagtaacgcc...
37,7290_RLTR4_MM-int,12083_RLTR4_MM-int,99.749572,tttggaggtc-ccaccgagatttggagacccctgcccagggaccac...
31,4128_RLTR4_MM-int,8324_RLTR4_MM-int,61.828358,tttggaggttccaccgagatcaggagacccctgcccagggaccac-...
27,4915_RLTR4_MM-int,9492_B2_Mm2,60.591133,gggccccttg-at-a-at-actctta-t--t-aa---tcct-act-...
41,8324_RLTR4_MM-int,7627_IAPEY4_I,60.155340,tgg-gcc-tgccttcgtctcccaggtaa--g-tcagtc-ggt---g...
34,4128_RLTR4_MM-int,7627_IAPEY4_I,58.059701,---t-ttg--ga--ggttccac-c-gagatcaggagacccctgccc...
3,9437_RLTR4_MM-int,4128_RLTR4_MM-int,56.180758,gagactgttggaccagggaatactggtaccctg-cc-ag-tccccc...
5,9437_RLTR4_MM-int,8324_RLTR4_MM-int,54.723032,gagactgttggaccagggaatactggtaccctgc-cagtcc-ccct...
8,9437_RLTR4_MM-int,7627_IAPEY4_I,51.399417,gagactgttggaccagggaatactggtaccctgccagtcccc-ctg...
4,9437_RLTR4_MM-int,7290_RLTR4_MM-int,44.418084,---g-ag-----act-g---tt-gga--cc-------aggga--a-...


### add additional information to the result df

In [87]:
adjusted_top10_result = add_seq_comparison_info( squire_top10_result, squire_sig_up, merged_result= True )

In [88]:
adjusted_top10_result.head()

,ID1,ID2,PercentIdentity,Alignment_str,seq1_len,seq2_len,total_match,adj_percentIdentity
0,9437_RLTR4_MM-int,11292_RLTR4_Mm,21.603499,gagactgttggaccagggaatactggtaccctgccagtccccctgg...,3430,741,741,35.531048
1,9437_RLTR4_MM-int,12455_RLTR4_Mm,21.603499,gagactgttggaccagggaatactggtaccctgccagtccccctgg...,3430,741,741,35.531048
2,9437_RLTR4_MM-int,4915_RLTR4_MM-int,5.918367,gagactgttggaccagggaatactggtaccctgccagtccccctgg...,3430,203,203,11.175337
3,9437_RLTR4_MM-int,4128_RLTR4_MM-int,56.180758,gagactgttggaccagggaatactggtaccctg-cc-ag-tccccc...,3430,2680,1927,63.076923
4,9437_RLTR4_MM-int,7290_RLTR4_MM-int,44.418084,---g-ag-----act-g---tt-gga--cc-------aggga--a-...,3430,7587,3370,61.178179


In [90]:
adjusted_top10_result.sort_values( by = 'adj_percentIdentity', ascending= False , inplace=True)

In [92]:
adjusted_top10_result.iloc[2:10]

,ID1,ID2,PercentIdentity,Alignment_str,seq1_len,seq2_len,total_match,adj_percentIdentity
27,4915_RLTR4_MM-int,9492_B2_Mm2,60.591133,gggccccttg-at-a-at-actctta-t--t-aa---tcct-act-...,203,180,123,64.229765
41,8324_RLTR4_MM-int,7627_IAPEY4_I,60.155340,tgg-gcc-tgccttcgtctcccaggtaa--g-tcagtc-ggt---g...,2575,2317,1549,63.327882
3,9437_RLTR4_MM-int,4128_RLTR4_MM-int,56.180758,gagactgttggaccagggaatactggtaccctg-cc-ag-tccccc...,3430,2680,1927,63.076923
31,4128_RLTR4_MM-int,8324_RLTR4_MM-int,61.828358,tttggaggttccaccgagatcaggagacccctgcccagggaccac-...,2680,2575,1657,63.063749
5,9437_RLTR4_MM-int,8324_RLTR4_MM-int,54.723032,gagactgttggaccagggaatactggtaccctgc-cagtcc-ccct...,3430,2575,1877,62.514571
34,4128_RLTR4_MM-int,7627_IAPEY4_I,58.059701,---t-ttg--ga--ggttccac-c-gagatcaggagacccctgccc...,2680,2317,1556,62.277366
8,9437_RLTR4_MM-int,7627_IAPEY4_I,51.399417,gagactgttggaccagggaatactggtaccctgccagtcccc-ctg...,3430,2317,1763,61.35375
4,9437_RLTR4_MM-int,7290_RLTR4_MM-int,44.418084,---g-ag-----act-g---tt-gga--cc-------aggga--a-...,3430,7587,3370,61.178179


### visualize sequence alignment

In [94]:
%%capture output
# I love this magic command 
for id1, id2, score, aln in adjusted_top10_result.iloc[2:10, 0:4].values:
    print( f"Comparing sequences of score {score:.2f}:\n\t{id1}\n\t{id2}")
    aln_ls = aln.split("\n")
    alignment_str_parser( aln_ls, line_width = 100, )
    
    

In [95]:
with open('./test_result/top10_seq_aln.txt', 'w') as f:
        f.write(output.stdout)